In [ ]:
# Data analysis modules: pandas, matplotlib, numpy, and etc.
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # render double resolution plot output for Retina screens 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Standard modules, MongoDB modules
import os, sys, json, datetime, pickle, multiprocessing, logging
from pprint import pprint

import pymongo
from pymongo import IndexModel, ASCENDING, DESCENDING

# Custom tool modules
import mongodb  # module for setting up connection with (local) MongoDB database
import multiprocessing_workers  # module for splitting workloads between processes
import utilities  # module for various custom utility functions
from config import * # import all global configuration variables

In [2]:
# read in method_1, method_2 results
m1_ibm_user_ids_lst = []
m2_ibm_user_ids_lst = []

if not m1_ibm_user_ids_lst:
    with open(M1_IBM_USER_IDS_PKL, 'rb') as f:
        m1_ibm_user_ids_lst = pickle.load(f)

if not m2_ibm_user_ids_lst:
    with open(M2_IBM_USER_IDS_PKL, 'rb') as f:
        m2_ibm_user_ids_lst = pickle.load(f)

In [3]:
# query only retweet tweets
match_dict = {'$match': {'quoted_status': {'$exists': True}}}

limit_dict = {'$limit': 100}

project_dict = {'$project': {'_id': 0,
                             'id': 1,
                             'retweet_count': 1,
                             'quoted_status.retweet_count': 1}}

# group_dict = {'$group': {'_id': '$user.id', # user's id
#                          'retweet_tweets_num': {'$sum': 1}, # total number of retweet tweets belongs to the user
#                          'retweet_tweets_total_retweet_count': {'$sum': '$retweet_count'} # sum of retweet counts of all retweet tweets belong to the user
#                          }}
# project_dict = {'$project': {'_id': 0,
#                             'user_id': '$_id',
#                             'retweet_tweets_num': 1,
#                             'retweet_tweets_total_retweet_count': 1}}

ppl_lst = [match_dict, project_dict, limit_dict]

updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)
cursor = updated_col.aggregate(pipeline=ppl_lst,
                               allowDiskUse=True) # Exceeded memory limit for $group, but didn't allow external sort. Pass allowDiskUse:true to opt in.
for doc in cursor:
    print(doc)

MongoDB on localhost:27017/tweets_ek.c2 connected successfully!
{'id': 838632147929923584, 'retweet_count': 0, 'quoted_status': {'retweet_count': 56}}
{'id': 838632255127891968, 'retweet_count': 0, 'quoted_status': {'retweet_count': 12}}
{'id': 838632334823985154, 'retweet_count': 0, 'quoted_status': {'retweet_count': 0}}
{'id': 838632128946503680, 'retweet_count': 3, 'quoted_status': {'retweet_count': 20}}
{'id': 838632575518277633, 'retweet_count': 1, 'quoted_status': {'retweet_count': 168}}
{'id': 838632878292492288, 'retweet_count': 26, 'quoted_status': {'retweet_count': 113}}
{'id': 838632847166550016, 'retweet_count': 0, 'quoted_status': {'retweet_count': 0}}
{'id': 838633114448592896, 'retweet_count': 0, 'quoted_status': {'retweet_count': 1}}
{'id': 838633132370890752, 'retweet_count': 0, 'quoted_status': {'retweet_count': 18}}
{'id': 838633585733165056, 'retweet_count': 0, 'quoted_status': {'retweet_count': 2}}
{'id': 838633769766510592, 'retweet_count': 0, 'quoted_status': {'r

In [2]:
# check how many tweets are 'retweet tweets'
updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

total = updated_col.count() # total number of tweets in database
count = updated_col.count(filter={'retweeted_status': {'$exists': True}})
print('{} ({:.2%}) tweets are "retweet tweets"'.format(count, (count / total)))

MongoDB on localhost:27017/tweets_ek.c2 connected successfully!
2465388 (48.88%) tweets are "retweet tweets"


In [4]:
# check how many 'retweet tweets' unique
updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

unique_retweet_tweet_id_set = set()

cursor = updated_col.find(filter={'retweeted_status': {'$exists': True}},
                          projection={'_id': 0, 'retweeted_status.id': 1})

for doc in cursor:
    unique_retweet_tweet_id = doc['retweeted_status']['id']
    unique_retweet_tweet_id_set.add(unique_retweet_tweet_id)
print('{} unique "retweet tweets"'.format(len(unique_retweet_tweet_id_set)))

MongoDB on localhost:27017/tweets_ek.c2 connected successfully!
555635 unique "retweet tweets"


In [5]:
# double check how many 'retweet tweets' unique
updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

unique_retweet_tweet_id_lst = []

unique_retweet_tweet_id_lst = updated_col.distinct('retweeted_status.id',
                                                   filter={'retweeted_status': {'$exists': True}})

print('{} unique "retweet tweets"'.format(len(unique_retweet_tweet_id_lst)))

MongoDB on localhost:27017/tweets_ek.c2 connected successfully!
555635 unique "retweet tweets"


In [6]:
# check how many authors of 'retweet tweets'
updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

retweet_tweet_user_id_set = set()

cursor = updated_col.find(filter={'retweeted_status': {'$exists': True}},
                          projection={'_id': 0, 'retweeted_status.user.id': 1})

for doc in cursor:
    retweet_tweet_user_id = doc['retweeted_status']['user']['id']
    retweet_tweet_user_id_set.add(retweet_tweet_user_id)
print('{} authors of "retweet tweets"'.format(len(retweet_tweet_user_id_set)))

MongoDB on localhost:27017/tweets_ek.c2 connected successfully!
143949 authors of "retweet tweets"


In [7]:
# double check how many 'retweet tweets' unique
updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

retweet_tweet_user_id_lst = []

retweet_tweet_user_id_lst = updated_col.distinct('retweeted_status.user.id',
                                                 filter={'retweeted_status': {'$exists': True}})

print('{} unique "retweet tweets"'.format(len(retweet_tweet_user_id_lst)))

MongoDB on localhost:27017/tweets_ek.c2 connected successfully!
143949 unique "retweet tweets"


In [3]:
"""
Check the correctness of update 'retweet_count' field manually in tweets_ek-2:tw_nt collection by comparing its distribution against
the tweets_ek:c2 collection
"""

import pandas as pd
import mongodb


'''
Extract all the "retweet_count" of native tweets in tweets_ek:c2 collection
'''
old_retweet_count_lst = []
print('Building old retweet_count list...')

tweets_ek_c2 = mongodb.initialize(db_name='tweets_ek', collection_name='c2')

cursor = tweets_ek_c2.find(filter={'retweeted_status': {'$exists': False}}, # native tweets only
                           projection={'_id': 0, 'retweet_count': 1})

for doc in cursor:
    retweet_count = int(doc['retweet_count'])
    old_retweet_count_lst.append(retweet_count)
print('List length: {}'.format(len(old_retweet_count_lst)))

'''
Extract all the "retweet_count" of native tweets in tweets_ek-2:tw_nt
'''
new_retweet_count_lst = []
print('Building new retweet_count list...')

tweets_ek_2_tw_nt = mongodb.initialize(db_name='tweets_ek-2', collection_name='tw_nt')

cursor = tweets_ek_2_tw_nt.find(projection={'_id': 0, 'retweet_count': 1}) # all tweets in this collection are native tweets
 
for doc in cursor:
    retweet_count = int(doc['retweet_count'])
    new_retweet_count_lst.append(retweet_count)
print('List length: {}'.format(len(new_retweet_count_lst)))

s1 = pd.Series(data=old_retweet_count_lst)
s2 = pd.Series(data=new_retweet_count_lst)

Building old retweet_count list...
MongoDB on localhost:27017/tweets_ek.c2 connected successfully!
List length: 2578199
Building new retweet_count list...
MongoDB on localhost:27017/tweets_ek-2.tw_nt connected successfully!
List length: 5812824


In [4]:
s1.describe()

count    2.578199e+06
mean     9.551819e-01
std      2.017118e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.749600e+04
dtype: float64

In [5]:
s2.describe()

count    5.812824e+06
mean     9.637095e-01
std      1.541557e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.845800e+04
dtype: float64